In [ ]:
import json
import os


class Product:
    
    def __init__(self, name, quantity, purchase_price, selling_price):
        
        """
        Initialize a Product instance with the provided attributes.

        Parameters:
        - name (str): The name of the product.
        - quantity (int): The quantity of the product.
        - purchase_price (float): The purchase price of the product.
        - selling_price (float): The selling price of the product.

        Returns: none.
        """
        
        self.name = name
        self.quantity = quantity
        self.purchase_price = purchase_price
        self.selling_price = selling_price
        self.quantity_sold = 0

    def sell(self, quantity):
        
        """
        Sell a specified quantity of the product.

        Parameters:
        - quantity (int): The quantity to sell.

        Returns:
        - bool: True if the sale was successful, False otherwise.
        """
        
        if self.quantity >= quantity:
            self.quantity -= quantity
            self.quantity_sold += quantity
            print(f"Vendita di {quantity} unità di {self.name}")
            return True
        else:
            print(f"Quantità non disponibile per {self.name}. Disponibile: {self.quantity}")
            return False

        
class Store:
    
    """
    Initialize a new Store instance and load products from files.
    """
    
    def __init__(self, file_name):
        self.file_name = file_name
        self.products = []
        if not os.path.exists(file_name):
            with open(file_name, "w") as file:
                pass
        self.load_products()

        
    def load_products(self):
        
        """
        Load products from JSON files.
        """
        
        try:
            with open(self.file_name, "r") as file:
                try:
                    data = json.load(file)
                    for product_data in data:
                        if 'name' in product_data and 'quantity' in product_data and 'purchase_price' in product_data and 'selling_price' in product_data:
                            self.products.append(Product(product_data['name'], product_data['quantity'], product_data['purchase_price'], product_data['selling_price']))
                        else:
                            print("Error in the product data format.")
                except json.JSONDecodeError:
                    pass
        except FileNotFoundError:
            pass

            
    def save_products(self):
        
        """
        Save products to JSON files and update "products.txt".
        """
    
        try:
            with open(self.file_name, "w") as file:
                data = [{'name': product.name, 'quantity': product.quantity, 'purchase_price': product.purchase_price, 'selling_price': product.selling_price} for product in self.products]
                json.dump(data, file)
        except IOError:
            print("Errore durante la scrittura del file")
        
        
    def add_product(self, name, quantity, purchase_price=None, selling_price=None):
        
        """
        
        Add or update a product in the Store.
        - If the product exists, update its quantity.
        - If it's a new product, ask for purchase and selling prices.
        
        """
        
        for p in self.products:
            if p.name == name:
                p.quantity += quantity
                print(f"Quantità di {name} aggiornata a {p.quantity}")
                self.save_products()
                return

        self.products.append(Product(name, quantity, purchase_price, selling_price))
        print(f"AGGIUNTO: {quantity} X {name}")
        self.save_products()
        
        
    def list_products(self):
        
        """
        Print the list of products in the warehouse.
        """
        
        print("PRODOTTO QUANTITA' PREZZO")
        for product in self.products:
            if product.quantity > 0:
                print(f"{product.name} {product.quantity} €{product.selling_price:.2f}")
            
    def record_sale(self):
        
        """
        Record a sale of products from the warehouse.
        """

        sale_items = []
        total_sales = 0
        total_gross_profits = 0
        total_net_profits = 0
        
        add_another_product = "si"
        while add_another_product.lower() == "si":
            product_name = input("Nome del prodotto: ")
            quantity_sold = None
            while quantity_sold is None:
                try:
                    quantity_sold = int(input("Quantità: "))
                except ValueError:
                    print("Quantità non valida. Si prega di inserire un numero intero.")
                    quantity_sold = None

            product_sold = None
            for product in self.products:
                if product.name == product_name:
                    product_sold = product
                    if product_sold.quantity >= quantity_sold:
                        product_sold.quantity -= quantity_sold
                        product_sold.quantity_sold += quantity_sold
                        unit_price = product_sold.selling_price
                        
                        total_price = quantity_sold * unit_price
                        sale_items.append({'name': product_sold.name, 'quantity': quantity_sold, 'unit_price': unit_price, 'total_price': total_price})
                        total_sales += total_price
                        total_gross_profits += quantity_sold * (unit_price - product.purchase_price)
                        total_net_profits += quantity_sold * (unit_price - product.purchase_price)
                    else:
                        print("Quantità non disponibile per la vendita")
                        product_sold = None
                    break

            if product_sold is None:
                print("Prodotto non trovato")
                continue
                
            add_another_product = input("Aggiungere un altro prodotto? (si/no): ")
    
        self.save_products()

        print("VENDITA REGISTRATA")
        for item in sale_items:
            print(f"- {item['quantity']} X {item['name']}: €{item['unit_price']:.2f}")
        print(f"Totale: €{total_sales:.2f}")

        
    def profits(self):
        
        """
        Calculate and print the total profits of the warehouse.
        """
        
        total_purchase = sum((product.purchase_price * product.quantity_sold) for product in self.products)
        total_sale = sum((product.selling_price * product.quantity_sold) for product in self.products)
    
        profit_gross = total_sale
        profit_net = profit_gross - total_purchase

        print(f"Profitto lordo: €{profit_gross:.2f}   Profitto netto: €{profit_net:.2f}")

        return profit_gross, profit_net

    
    def help(self):
        
        """
        Shows the available commands.
        """
        
        print("I comandi disponibili sono i seguenti:")
        print("aggiungi: aggiungi un prodotto al magazzino")
        print("elenca: elenca i prodotto in magazzino")
        print("vendita: registra una vendita effettuata")
        print("profitti: mostra i profitti totali")
        print("aiuto: mostra i possibili comandi")
        print("chiudi: esci dal programma")

        
    def close(self):
        
        """
        Close the program
        """
        
        self.save_products()
        print("Bye bye")

store = Store("products.txt")

while True:
    command = input("Inserisci un comando: ")
    if command == "aggiungi":
        name = input("Nome del prodotto: ")
        
        quantity = None
        while quantity is None:
            try:
                quantity = int(input("Quantità: "))
                if quantity < 0:
                    print("La quantità non può essere negativa. Si prega di inserire un numero positivo.")
                    quantity = None
            except ValueError:
                print("Quantità non valida. Si prega di inserire un numero intero.")

        existing_product = any(product.name == name for product in store.products)
        
        purchase_price = selling_price = None
        if not existing_product:
            while purchase_price is None or purchase_price < 0:
                try:
                    purchase_price = float(input("Prezzo di acquisto: "))
                    if purchase_price < 0:
                        print("Il prezzo di acquisto non può essere negativo.")
                except ValueError:
                    print("Prezzo di acquisto non valido. Si prega di inserire un numero.")
            
            while selling_price is None or selling_price < 0:
                try:
                    selling_price = float(input("Prezzo di vendita: "))
                    if selling_price < 0:
                        print("Il prezzo di vendita non può essere negativo.")
                except ValueError:
                    print("Prezzo di vendita non valido. Si prega di inserire un numero.")

            store.add_product(name, quantity, purchase_price, selling_price)
        else:
            store.add_product(name, quantity)
    
    elif command == "elenca":
        store.list_products()
    elif command == "vendita":
        store.record_sale()
    elif command == "profitti":
        store.profits()
    elif command == "aiuto":
        store.help()
    elif command == "chiudi":
        store.close()
        break
    else:
        print("Comando non disponibile")
        print("I comandi disponibili sono i seguenti:")
        print("aggiungi: aggiungi un prodotto al magazzino")
        print("elenca: elenca i prodotto in magazzino")
        print("vendita: registra una vendita effettuata")
        print("profitti: mostra i profitti totali")
        print("aiuto: mostra i possibili comandi")
        print("chiudi: esci dal programma")